In [12]:
import math
import random
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

# Attempt to use qBraid, otherwise fall back to local Aer simulator
try:
    from qbraid import get_device
    USE_QBRAID = True
except ImportError:
    from qiskit_aer import AerSimulator
    USE_QBRAID = False
    print("qBraid not found, using local Aer simulator.")

# --- Configuration ---
SENSOR_DATA = 32191 # Dummy sensor data for signing

# --- Trusted Authority: Secret Key Generator ---
class SKG:
    """A trusted entity to generate and store user secrets."""
    def __init__(self):
        self.user_registry = {} # Stores keys and secrets for each user
        print("SKG instance created.")

    def register_user(self, user_id: str, key: list, secret_angle: float):
        """Manually stores a user's key and secret angle."""
        self.user_registry[user_id] = {'key': key, 'phi': secret_angle}
        print(f"User '{user_id}' registered with SKG.")

# --- Quantum Protocols ---
def bb84_protocol(num_bits=200) -> list:
    """
    A simplified simulation of the BB84 protocol to generate a shared secret key.
    Returns the sifted key.
    """
    # Alice generates random bits and bases (0=Z, 1=X)
    alice_bits = np.random.randint(2, size=num_bits)
    alice_bases = np.random.randint(2, size=num_bits)

    # Bob generates his random bases for measurement
    bob_bases = np.random.randint(2, size=num_bits)

    # Alice prepares and sends qubits
    qc = QuantumCircuit(num_bits, num_bits)
    for i in range(num_bits):
        if alice_bits[i] == 1:
            qc.x(i)
        if alice_bases[i] == 1:
            qc.h(i)
    qc.barrier()

    # Bob measures the incoming qubits
    for i in range(num_bits):
        if bob_bases[i] == 1:
            qc.h(i)
        qc.measure(i, i)

    # Execute the circuit
    if USE_QBRAID:
        backend = get_device("local_qiskit_simulator")
    else:
        backend = AerSimulator()

    counts = backend.run(qc, shots=1).result().get_counts(qc)
    bob_results = [int(bit) for bit in list(counts.keys())[0]][::-1]

    # Sifting: Alice and Bob compare bases and keep matching bits
    sifted_key = []
    for i in range(num_bits):
        if alice_bases[i] == bob_bases[i]:
            sifted_key.append(alice_bits[i])

    # Basic check for eavesdropping (QBER)
    matching_bases_indices = np.where(alice_bases == bob_bases)[0]
    errors = np.sum(np.array(alice_bits)[matching_bases_indices] != np.array(bob_results)[matching_bases_indices])
    qber = errors / len(sifted_key) if len(sifted_key) > 0 else 0

    if qber > 0.1: # High error rate might mean an eavesdropper
        print(f"Warning: High QBER of {qber:.2f} detected. Key may be compromised.")
        return []

    print(f"BB84 completed. Generated key with {len(sifted_key)} bits (QBER: {qber:.2f}).")
    return sifted_key

def apply_qotp(qc: QuantumCircuit, target_qubits: range, key: list):
    """Encrypts qubits using the Quantum One-Time Pad."""
    key_len_needed = 2 * len(target_qubits)
    if len(key) < key_len_needed:
        raise ValueError(f"QOTP key is too short. Needs {key_len_needed}, got {len(key)}.")

    for i, qubit_idx in enumerate(target_qubits):
        if key[2 * i] == 1: qc.z(qubit_idx)
        if key[2 * i + 1] == 1: qc.x(qubit_idx)

def apply_qotp_inverse(qc: QuantumCircuit, target_qubits: range, key: list):
    """Decrypts qubits by applying the inverse QOTP operation."""
    key_len_needed = 2 * len(target_qubits)
    if len(key) < key_len_needed:
        raise ValueError(f"QOTP key is too short. Needs {key_len_needed}, got {len(key)}.")
        
    for i, qubit_idx in enumerate(target_qubits):
        # Inverse is applied in reverse order
        if key[2 * i + 1] == 1: qc.x(qubit_idx)
        if key[2 * i] == 1: qc.z(qubit_idx)

# --- QIBS Core Logic ---
def create_signature(message_binary: str, sender_id: str, key: list, phi_angle: float):
    """
    Alice's signing process.
    Creates the original message state |P> and the signed state |S>.
    """
    num_qubits = len(message_binary)

    # 1. Prepare the message state |P>
    qc_P = QuantumCircuit(num_qubits, name="P_state")
    for i, bit in enumerate(message_binary):
        if bit == '1':
            qc_P.x(i)

    # 2. Create the signature circuit |S> by transforming and encrypting |P>
    qc_S = qc_P.copy(name="S_state")
    # Apply the unitary transformation
    for i in range(num_qubits):
        qc_S.u(math.pi / 2, phi_angle, 0, i)
    # Encrypt with QOTP
    apply_qotp(qc_S, range(num_qubits), key)

    print(f"Signature created by '{sender_id}' for message |{message_binary}⟩.")
    return qc_P, qc_S

def verify_signature(qc_P, qc_S, signer_id: str, skg: SKG) -> bool:
    """
    Bob and SKG's verification process.
    SKG uses its records to decrypt |S> and checks if it matches |P>.
    """
    # Retrieve signer's info from the trusted SKG
    if signer_id not in skg.user_registry:
        print(f"Verification failed: Signer '{signer_id}' not found in registry.")
        return False
    
    signer_info = skg.user_registry[signer_id]
    retrieved_key = signer_info['key']
    retrieved_phi = signer_info['phi']

    # SKG attempts to recover the original state |P'> from the signature |S>
    qc_recovered = qc_S.copy(name="recovered_P")
    # Decrypt with QOTP
    apply_qotp_inverse(qc_recovered, range(qc_S.num_qubits), retrieved_key)
    # Apply the inverse unitary transformation
    for i in range(qc_S.num_qubits):
        qc_recovered.u(math.pi / 2, retrieved_phi, 0, i).inverse()

    # Compare the statevectors of the original and recovered circuits
    original_sv = Statevector(qc_P)
    recovered_sv = Statevector(qc_recovered)
    
    is_valid = original_sv.equiv(recovered_sv)
    
    if is_valid:
        print("Verification successful: Recovered state matches original.")
    else:
        print("Verification FAILED: States do not match. Signature is invalid.")
        
    return is_valid

# --- Main Execution ---
if __name__ == "__main__":
    print("\n--- Quantum Identity-Based Signature (QIBS) Simulation ---")

    # 1. Initialization
    skg = SKG()
    ALICE_ID = "Carbon_Sensor_7"
    BOB_ID = "Regulator_Agency"
    
    # Here, we generate one key and share it for simplicity.
    shared_key = bb84_protocol()
    
    # Convert sensor data to a binary string message
    message = bin(SENSOR_DATA)[2:]

    if len(shared_key) < 2 * len(message):
        print(f"\nError: Key length ({len(shared_key)}) is insufficient for message length ({len(message)}).")
        print("Aborting simulation. Try running again to get a longer key.")
        exit()

    alice_secret_phi = random.uniform(0, 2 * math.pi)
    skg.register_user(ALICE_ID, shared_key, alice_secret_phi)
    skg.register_user(BOB_ID, shared_key, 0) # Bob doesn't need a secret angle
    print("-" * 50)

    # 2. Alice signs the message
    qc_P, qc_S = create_signature(message, ALICE_ID, shared_key, alice_secret_phi)
    print("-" * 50)
    
    # 3. Bob receives the signature package (|P> and |S>) and verifies it with the SKG
    print("Bob is now verifying the received signature...")
    is_valid_signature = verify_signature(qc_P, qc_S, ALICE_ID, skg)
    print(f"Final Result: The signature is {'VALID' if is_valid_signature else 'INVALID'}.")
    print("-" * 50)

    # 4. Fraud Test: Tamper with the signature and re-verify
    print("\n--- Tampering Test ---")
    print("An attacker intercepts |S> and applies a random X-gate.")
    qc_S_tampered = qc_S.copy()
    tamper_qubit_index = random.randint(0, len(message) - 1)
    qc_S_tampered.x(tamper_qubit_index)
    
    print("Bob is verifying the tampered signature...")
    is_tampered_valid = verify_signature(qc_P, qc_S_tampered, ALICE_ID, skg)
    
    if not is_tampered_valid:
        print("Test Result: PASSED. The tampered signature was correctly rejected.")
    else:
        print("Test Result: FAILED. The system accepted a forged signature.")
    print("-" * 50)

qBraid not found, using local Aer simulator.

--- Quantum Identity-Based Signature (QIBS) Simulation ---
SKG instance created.
BB84 completed. Generated key with 103 bits (QBER: 0.00).
User 'Carbon_Sensor_7' registered with SKG.
User 'Regulator_Agency' registered with SKG.
--------------------------------------------------
Signature created by 'Carbon_Sensor_7' for message |111110110111111⟩.
--------------------------------------------------
Bob is now verifying the received signature...
Verification successful: Recovered state matches original.
Final Result: The signature is VALID.
--------------------------------------------------

--- Tampering Test ---
An attacker intercepts |S> and applies a random X-gate.
Bob is verifying the tampered signature...
Verification FAILED: States do not match. Signature is invalid.
Test Result: PASSED. The tampered signature was correctly rejected.
--------------------------------------------------
